## Initiation au CNN avec Pytorch

Ce TP s'effectue individuellement ou en binome. Veuillez respecter les consignes suivantes pour le rendu de votre travail :

* Renommez le selon le format suivant : "DL_2023_TP_Torch_prenom1_nom1_prenom2_nom2.ipynb".
* Veillez à ce que votre nom et prénom soient complétés dans la cellule ci-dessous.
* Veillez à avoir bien exécuté toutes les cellules de code et que les résultats soient tous bien visible dans le notebook sans nécessiter une ré-exécution.
* Partagez le notebook avec hana.sebia@univ-lyon1.fr

Veuillez compléter vos noms et prénoms ci-dessous :

*   **Prenom 1** : ...
*   **Nom 1** : ...
*   **Prenom 2** : ...
*   **Nom 2** : ...

Ce TP est une introduction au framework Pytorch. Nous allons construire une des premières architectures de CNN présenté par [Yann Le Cun](https://fr.wikipedia.org/wiki/Yann_Le_Cun), un [LeNet](http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf).

L'architecture du LeNet est détaillée dans la figure ci-dessous:


![leNet5.jpeg](leNet5.jpeg "Architecture Lenet")

In [1]:
# on importe les bibliothèques pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(42)

In [10]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
    x = torch.ones(1, device=device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


### Chargement du jeu de données
La tâche que nous souhaitons réaliser est la classification d'image [MNIST](https://fr.wikipedia.org/wiki/Base_de_donn%C3%A9es_MNIST). La base de données MNIST (Modified National Institute of Standards and Technology) est une base de données de chiffres écrits à la main. C'est un jeu de données très utlisé en apprentissage automatique. Il regroupe 60000 images d'apprentissage et 10000 images de test. On peut télécharger ces données à partir du module dataset de torchvision en séparant le chargement du train/test set. Il est également possible d'appliquer un ensemble de transformations aux images dès le chargement.

In [3]:
from torchvision import datasets, transforms # On peut importer directement le dataset de pytorch

# On définit transforms qui permet de redimensionner l'image en 32*32 et de la transformer en tensor
transforms = transforms.Compose([transforms.Resize((32, 32)),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5,), (0.5,))])

# On télécharge et on créer la dataset d'entraienement à l'aide du module datasets de torchvision
train_dataset = datasets.MNIST(root='mnist_data', 
                               train=True, 
                               transform=transforms,
                               download=True)

# On télécharge et on créer la dataset de test à l'aide du module datasets de torchvision
valid_dataset = datasets.MNIST(root='mnist_data', 
                               train=False, 
                               transform=transforms)

Une fois les train/test sets chargés, on définit des dataloaders qui permettent de créer des batchs pour la phase train de l'apprentissage de notre modèle comme suit :

In [15]:
from torch.utils.data import DataLoader 

BATCH_SIZE = 32 #taille du batch size


# On définit le data loaders d'entraienement . Le data loaders permet de créer des batchs. On doit lui renseigner le batch size.
train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=True)
# On définit le data loaders de validation . 
valid_loader = DataLoader(dataset=valid_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=False)

### Définition du modèle

---
<span style='color:Green'>**Question**</span> 9 points

Implémenter la classe LeNet avec l'architecture proposée en utilisant l’interface [nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) de PyTorch:

---


Dans l'initialisation de la classe LeNet
1. La première couche [convolutive](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html) avec 6 noyaux de taille 5×5 et le stride de 1.
2. Une couche de [sous-échantillonnage/mise](https://pytorch.org/docs/stable/generated/torch.nn.AvgPool2d.html) en commun avec 6 noyaux de taille 2×2.
3. La deuxième couche convolutive avec la même configuration que la première, cette fois avec 16 filtres. La sortie de cette couche est de 10×10×16.
4. La deuxième couche de mise en commun. La logique est identique à celle de la précédente, mais cette fois, la couche comporte 16 filtres. La sortie de cette couche est de taille 5×5×16.
5. La dernière couche convolutive avec 120 noyaux 5×5.
6. La dernière couche est un réseau de [neurones simple](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)

Dans la fonction forward de la classe LeNet définissez le passage de la donnée x en appliquant à la fin un [softmax](https://pytorch.org/docs/stable/generated/torch.nn.functional.softmax.html) pour calculer la probabilité d'appartenance à la classe des chiffres de mnist.


**Indice**
Indice: l’utilisation de la méthode [.view()](https://pytorch.org/docs/stable/generated/torch.Tensor.view.html) ou de la couche [nn.Flatten()](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html) peut être utile pour ré-arranger les tenseurs avant ou après les couches linéaires. Par exemple, x.view(-1, 1, 28, 28) permet de transformer un tenseur de dimensions 784 en un tenseur de dimensions (batch, 1, 28, 28)…

In [5]:

class LeNet(nn.Module): # On créer la classe LeNet qui hérite de la classe mère Module

    def __init__(self): # On définit 
        super(LeNet, self).__init__()
        
        self.feature_extractor = nn.Sequential(  
            # Couche 1 (C1) : La première couche convolutive avec 6 noyaux de taille 5×5 et le stride de 1. 
            # Étant donné la taille de l'entrée (32×32×1), la sortie de cette couche est de taille 28×28×6.
            # kernel
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),  # 1.5 points
            nn.BatchNorm2d(6),
            nn.ReLU(),
            
            #Couche 2 (S2) : Une couche de sous-échantillonnage/mise en commun avec 6 noyaux de taille 2×2.
            nn.AvgPool2d(kernel_size=2),  # 0.5 points
            
            
            #Couche 3 (C3) : La deuxième couche convolutive avec la même configuration que la première, cependant, 
            #cette fois avec 16 filtres. La sortie de cette couche est de 10×10×16.
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1), # 1.5 points
            nn.BatchNorm2d(16),
            nn.ReLU(),
            
            #Couche 4 (S4) : La deuxième couche de mise en commun. La logique est identique à celle de la précédente, 
            #mais cette fois, la couche comporte 16 filtres. La sortie de cette couche est de taille 5×5×16.
            nn.AvgPool2d(kernel_size=2),  # 0.5 points
            
            #Couche 5 (C5) : La dernière couche convolutive avec 120 noyaux 5×5. 
            #Étant donné que l'entrée de cette couche est de taille 5×5×16 et que les noyaux sont de taille 5×5, 
            #la sortie est 1×1×120. Par conséquent, les couches S4 et C5 sont entièrement connectées. 
            #C'est aussi pourquoi dans certaines implémentations de LeNet-5, on utilise une couche entièrement connectée au lieu d'une couche convolutive comme 5ème couche. 
            #La raison pour laquelle cette couche reste une couche convolutive est le fait que si l'entrée du réseau est plus grande que celle utilisée dans l'entrée initiale (donc 32×32 dans ce cas), 
            #cette couche ne sera pas une couche entièrement connectée, car la sortie de chaque noyau ne sera pas 1×1.
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1), # 1.5 points
            nn.BatchNorm2d(120),
            nn.ReLU()
        )
        
         ## La dernière couche est un réseau de neurones simple # 1.5 points
        self.classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84),
            nn.ReLU(),
            nn.Linear(in_features=84, out_features=10),
        )

    def forward(self, x): # on défini le passage de nos données # 2 points
        
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probs = F.softmax(logits, dim=1)
        return logits, probs
        

net = LeNet()
print(net) # On peut afficher les paramètres du modèle

LeNet(
  (feature_extractor): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (4): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (8): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
    (9): BatchNorm2d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
  )
  (classifier): Sequential(
    (0): Linear(in_features=120, out_features=84, bias=True)
    (1): ReLU()
    (2): Linear(in_features=84, out_features=10, bias=True)
  )
)


---
<span style='color:Green'>**Question**</span> 3 points

Compléter et commenter la fonction train(chaque ligne du code), celle-ci permet d'entrainer votre modèle:

---

In [6]:
# On créer la fonction qui permet d'entrainer le modèle 

def train(train_loader, model, criterion, optimizer):
    '''
    Function for the training step of the training loop
    '''
    model.train() # on passe le modèle en entrainement
    running_loss = 0
    
    for X, y_true in train_loader:# on itére sur les données d'entrainement 

        optimizer.zero_grad() # on initialise l'erreur du gradient à zéro
        
        X = X.to(device) # on envoie les données X sur la GPU # 0.5 points
        y_true = y_true.to(device) # on envoie les données Y sur la GPU # 0.5 points
    
        # Forward pass (on passe les données dans le modèle) # 1 points
        y_hat, _ = model(X)  
        loss = criterion(y_hat, y_true) # On calcul l'erreur du modèle avec la loss choisie
        # Rétropropagation du gradient
        loss.backward() # 0.5 points
        optimizer.step() # Descente de gradient (une itération) # 0.5 points
        running_loss += loss.item()
        
    running_loss /= len(train_loader)
    return model, optimizer, running_loss

---
<span style='color:Green'>**Question**</span> 2 points

En vous inspirant de la fonction train, completer la fonction validate, celle-ci permet de tester votre modèle:

---

In [7]:
# On créer la fonction qui permet de valider le modèle 

def validate(valid_loader, model, criterion):
    '''
    Function for the validation step of the training loop
    '''
   
    model.eval()
    running_loss = 0
    
    for X, y_true in valid_loader:
    
        X = X.to(device)  # 0.5 points
        y_true = y_true.to(device) # 0.5 points

        # Forward pass and record loss
        y_hat, _ = model(X)   # 0.5 points
        loss = criterion(y_hat, y_true)  # 0.5 points
        running_loss += loss.item()

    running_loss /= len(valid_loader)
    return model, running_loss

---
<span style='color:Green'>**Question**</span> 3 points

Ecrivez la fonction training_loop qui prend en paramètre le model, le criterion, l'optimizer, le train_loder, le valid_loader et le nombre d'épochs. Cette fonction permet de faire une étape de train et une étape de validate par epoch. Affichez l'erreur d'apprentissage et de validation toutes les 5 épochs. 

---
--- 
**Note**
Lors de la validation, les gradients ne doivent pas être modifiés ([.no_grad](https://pytorch.org/docs/stable/generated/torch.no_grad.html)).

---

In [8]:
def training_loop(model, criterion, optimizer, train_loader, valid_loader, epochs, print_every=1):
    
    # Train model
    for epoch in range(0, epochs): # 0.5 points
        train_loss = 0 
        valid_loss = 0
        
        # training
        model, optimizer, train_loss = train(train_loader, model, criterion, optimizer) # 1 point
    

        # validation
        with torch.no_grad(): # 
            model, valid_loss = validate(valid_loader, model, criterion) # 1 point
    

        if epoch % print_every == (print_every - 1): # 0.5 point
            print("epoch",epoch)
            print("Train loss",train_loss)
            print("Valid loss",valid_loss)
    
    return model, optimizer

---
<span style='color:Green'>**Question**</span> 1 point

Commenter les lignes suivantes, celles-ci permettent de définir votre optimizer et votre fonction perte qui permet de calculer l'erreur de votre modèle;

---

In [11]:
model = net.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

---
<span style='color:Green'>**Question**</span> 2 points

Lancez l'entrainement de votre modèle en choisissant un nombre d'epoch judicieusement.

---

In [12]:
N_EPOCHS = 100
model, optimizer = training_loop(model, criterion, optimizer, train_loader, valid_loader, N_EPOCHS)

epoch 0
Train loss 0.1357722154661392
Valid loss 0.04824028378019642
epoch 1
Train loss 0.05375609795760053
Valid loss 0.03744413881315686
epoch 2
Train loss 0.042262281642792124
Valid loss 0.029126564432891418
epoch 3
Train loss 0.03581972745487777
Valid loss 0.023513364231870112
epoch 4
Train loss 0.03019797623597551
Valid loss 0.026780237271057373
epoch 5
Train loss 0.025854463253624272
Valid loss 0.02585894377495632
epoch 6
Train loss 0.025474869787559147
Valid loss 0.031635809375223235
epoch 7
Train loss 0.021368482567963656
Valid loss 0.026280168384807973
epoch 8
Train loss 0.019090564645741444
Valid loss 0.029178795483157622
epoch 9
Train loss 0.01821924878869225
Valid loss 0.02836413641415865
epoch 10
Train loss 0.01634208109034565
Valid loss 0.02935498232756935
epoch 11
Train loss 0.015046922099643173
Valid loss 0.03147459590431849
epoch 12
Train loss 0.015221643221531848
Valid loss 0.02511279735062672
epoch 13
Train loss 0.012842210487065797
Valid loss 0.03847892247290383
epo